In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from pandas_datareader import data
from datetime import timedelta, date
import numpy_financial as npf
import yfinance as yf

In [2]:
#URL
url_financials_is = 'https://finance.yahoo.com/quote/{}/financials?p={}'

#Variable
ticker_code = 'BBCA' #aapl= 229.69779556090393
negara = 'indonesia'
if negara == 'indonesia':
    ticker_code = ticker_code + '.JK'

#headers
headers = { 'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36' }
response = requests.get(url_financials_is.format(ticker_code, ticker_code),headers={'user-agent':'my-app'})

#scriping
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

#location variable of yahoo
I_S_start = json_data['context']['dispatcher']['stores']

ticker_code

'BBCA.JK'

In [3]:
url_financials_fcf = 'https://finance.yahoo.com/quote/{}/cash-flow?p={}'

response_fcf = requests.get(url_financials_fcf.format(ticker_code, ticker_code),headers={'user-agent':'my-app'})
soup_fcf = BeautifulSoup(response_fcf.text, 'html.parser')
script_data_fcf =  soup_fcf.find('script', text=pattern).contents[0]
start_fcf = script_data_fcf.find("context")-2
json_data_fcf = json.loads(script_data_fcf[start_fcf:-12])

fcf_start = json_data_fcf['context']['dispatcher']['stores']

In [4]:
def date_is_annual(input1, I_S_start, ticker_code):
    if input1 == 'lastyear':
        year = 0
        input2 = -1
    elif input1 == 'yearago':
        year = 1
        input2 = -2
    elif input1 ==  'years2ago':
        year = 2
        input2 = -3
    elif input1 ==  'years3ago':
        year = 3
        input2 = -4
    elif input1 ==  'years4ago':
        year = 4
        input2 = -5
    else:
        print('out of index')
    #old
    date = I_S_start['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory'][year]['endDate']['fmt']
    ###reportedCurrency = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualOperatingExpense'][input2]['currencyCode']
    revenue = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualTotalRevenue'][input2]['reportedValue']['raw']
    try:
        EBITDA = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualNormalizedEBITDA'][input2]['reportedValue']['raw']
    except:
        EBITDA = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualReconciledDepreciation'][input2]['reportedValue']['raw'] + I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualInterestExpense'][input2]['reportedValue']['raw'] + I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualPretaxIncome'][input2]['reportedValue']['raw'] + I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualMinorityInterests'][input2]['reportedValue']['raw']
        
    depreciation = I_S_start['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements'][year]['depreciation']['raw']
    ebitda_persen = EBITDA/revenue
    ebit = EBITDA - depreciation
    ebit_persen = ((EBITDA - depreciation)/revenue)
    depreciation_persen = ebitda_persen - ebit_persen
    
    
    
    
    #variable
    date = date
    income_statement = {}
    income_statement[input1] = {}
    income_statement[input1]['revenue'] = revenue
    income_statement[input1]['EBITDA'] = EBITDA
    income_statement[input1]['EBITDA(%)'] = ebitda_persen
    income_statement[input1]['Ebit'] = ebit
    income_statement[input1]['Ebit(%)'] = ebit_persen
    income_statement[input1]['Depreciation'] = depreciation
    income_statement[input1]['Depreciation(%)'] = depreciation_persen
    
    
    
    return income_statement

In [5]:
income_statement_3 = date_is_annual('lastyear', I_S_start, ticker_code)
income_statement_2 = date_is_annual('yearago', I_S_start, ticker_code)
income_statement_1 = date_is_annual('years2ago', I_S_start, ticker_code)

In [6]:
cagr_revenue = ((income_statement_3['lastyear']['revenue']/income_statement_1['years2ago']['revenue'])**(1/3)-1)

revenue4 = (income_statement_3['lastyear']['revenue']*cagr_revenue) + income_statement_3['lastyear']['revenue']
revenue5 = (revenue4*cagr_revenue) + revenue4
revenue6 = (revenue5*cagr_revenue) + revenue5

revenue7 = (revenue6*cagr_revenue) + revenue6

In [7]:
ebitda3_persen = income_statement_3['lastyear']['EBITDA(%)']
ebitda2_persen = income_statement_2['yearago']['EBITDA(%)']
ebitda1_persen = income_statement_1['years2ago']['EBITDA(%)']

rata2ebitda = (ebitda1_persen + ebitda2_persen + ebitda3_persen)/3

ebitda3 = income_statement_3['lastyear']['EBITDA']
ebitda2= income_statement_2['yearago']['EBITDA']
ebitda1 = income_statement_1['years2ago']['EBITDA']
ebitda4 = revenue4 * rata2ebitda
ebitda5 = revenue5 * rata2ebitda
ebitda6 = revenue6 * rata2ebitda



In [8]:
ebit3_persen = income_statement_3['lastyear']['Ebit(%)']
ebit2_persen = income_statement_2['yearago']['Ebit(%)']
ebit1_persen = income_statement_1['years2ago']['Ebit(%)']
rata2ebit = (ebit1_persen + ebit2_persen + ebit3_persen)/3



In [9]:
rata2Depreciation = rata2ebitda - rata2ebit

Depreciation4 = revenue4 * rata2Depreciation
Depreciation5 = revenue5 * rata2Depreciation
Depreciation6 = revenue6 * rata2Depreciation


In [10]:
ebit4 = ebitda4 - Depreciation4
ebit5 = ebitda5 - Depreciation5
ebit6 = ebitda6 - Depreciation6


In [11]:
income_statement_4 = {
    'future1': {
        'revenue': revenue4, 
        'EBITDA' : ebitda4, 
        'EBITDA(%)' : rata2ebitda, 
        'Ebit' : ebit4, 
        'Ebit(%)' : rata2ebit, 
        'Depreciation' : Depreciation4, 
        'Depreciation(%)' : rata2Depreciation
    }
}

income_statement_5 = {
    'future2': {
        'revenue': revenue5, 
        'EBITDA' : ebitda5, 
        'EBITDA(%)' : rata2ebitda, 
        'Ebit' : ebit5, 
        'Ebit(%)' : rata2ebit, 
        'Depreciation' : Depreciation5, 
        'Depreciation(%)' : rata2Depreciation
    }
}

income_statement_6 = {
    'future3': {
        'revenue': revenue6, 
        'EBITDA' : ebitda6, 
        'EBITDA(%)' : rata2ebitda, 
        'Ebit' : ebit6, 
        'Ebit(%)' : rata2ebit, 
        'Depreciation' : Depreciation6, 
        'Depreciation(%)' : rata2Depreciation
    }
}



In [12]:
income_statement = income_statement_1 | income_statement_2 | income_statement_3 | income_statement_4 | income_statement_5 | income_statement_6
income_statement = pd.DataFrame.from_dict(income_statement,orient='columns')
income_statement

,years2ago,yearago,lastyear,future1,future2,future3
revenue,7.296834e+13,7.670644e+13,8.013998e+13,8.268387e+13,8.530851e+13,8.801647e+13
EBITDA,5.029426e+13,4.551218e+13,4.881010e+13,5.213635e+13,5.379132e+13,5.549883e+13
EBITDA(%),6.892614e-01,5.933293e-01,6.090606e-01,6.305505e-01,6.305505e-01,6.305505e-01
Ebit,4.860038e+13,4.327170e+13,4.667199e+13,4.995620e+13,5.154196e+13,5.317807e+13
Ebit(%),6.660476e-01,5.641208e-01,5.823809e-01,6.041831e-01,6.041831e-01,6.041831e-01
Depreciation,1.693877e+12,2.240482e+12,2.138107e+12,2.180154e+12,2.249359e+12,2.320761e+12
Depreciation(%),2.321386e-02,2.920852e-02,2.667966e-02,2.636735e-02,2.636735e-02,2.636735e-02


In [13]:
url_balenceSheet = 'https://finance.yahoo.com/quote/{}/balance-sheet?p={}'

response_bs = requests.get(url_balenceSheet.format(ticker_code, ticker_code),headers={'user-agent':'my-app'})

#scriping
soup_bs = BeautifulSoup(response_bs.text, 'html.parser')
pattern_bs = re.compile(r'\s--\sData\s--\s')
script_data_bs = soup_bs.find('script', text=pattern_bs).contents[0]
start_bs = script_data_bs.find("context")-2
json_data_bs = json.loads(script_data_bs[start:-12])

#location variable of yahoo
B_S_start = json_data_bs['context']['dispatcher']['stores']


In [14]:
def date_bs_annual(input1, B_S_start, income_statement, ticker_code):
    if input1 == 'lastyear':
        year = 0
        input2 = -1
    elif input1 == 'yearago':
        year = 1
        input2 = -2
    elif input1 ==  'years2ago':
        year = 2
        input2 = -3
    elif input1 ==  'years3ago':
        year = 3
        input2 = -4
    elif input1 ==  'years4ago':
        year = 4
        input2 = -5
    else:
        print('out of index')
    
    try:
        total_cash = B_S_start['QuoteTimeSeriesStore']['timeSeries']['annualCashCashEquivalentsAndShortTermInvestments'][input2]['reportedValue']['raw']
    except:
        total_cash = B_S_start['QuoteTimeSeriesStore']['timeSeries']['annualCashAndCashEquivalents'][input2]['reportedValue']['raw']
    revenue =  income_statement[input1]['revenue']
    account_receivables = B_S_start['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements'][year]['netReceivables']['raw']
    inventories = B_S_start['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements'][year]['inventory']['raw']
    accountsPayable = B_S_start['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements'][year]['accountsPayable']['raw']
    capitalExpenditure = B_S_start['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements'][year]['capitalExpenditures']['raw']

    
    balance_sheet = {}
    balance_sheet[input1] = {}
    balance_sheet[input1]['Total Cash'] = total_cash
    balance_sheet[input1]['Total Cash(%)'] = total_cash / revenue
    balance_sheet[input1]['account receivables'] = account_receivables
    balance_sheet[input1]['account receivables(%)'] = account_receivables / revenue
    balance_sheet[input1]['Inventories'] = inventories
    balance_sheet[input1]['Inventories(%)'] = inventories / revenue
    balance_sheet[input1]['accounts Payable'] = accountsPayable
    balance_sheet[input1]['accounts Payable(%)'] = accountsPayable / revenue
    balance_sheet[input1]['capital Expenditure'] = capitalExpenditure
    balance_sheet[input1]['capital Expenditure(%)'] = capitalExpenditure / revenue
    
    return balance_sheet


In [15]:
bs1 = date_bs_annual('years2ago', B_S_start, income_statement, ticker_code)
bs2 = date_bs_annual('yearago', B_S_start, income_statement, ticker_code)
bs3 = date_bs_annual('lastyear', B_S_start, income_statement, ticker_code)

In [16]:
cash1_persen = bs1['years2ago']['Total Cash(%)']
cash2_persen = bs2['yearago']['Total Cash(%)']
cash3_persen = bs3['lastyear']['Total Cash(%)']
rata2cash = (cash1_persen + cash2_persen + cash3_persen)/3

cash4 = revenue4 * rata2cash
cash5 = revenue5 * rata2cash
cash6 = revenue6 * rata2cash

In [17]:
account_receivables_1_persen = bs1['years2ago']['account receivables(%)']
account_receivables_2_persen = bs2['yearago']['account receivables(%)']
account_receivables_3_persen = bs3['lastyear']['account receivables(%)']
rata_AR_cash = (account_receivables_1_persen + account_receivables_2_persen + account_receivables_3_persen)/3

account_receivables4 = revenue4 * rata_AR_cash
account_receivables5 = revenue5 * rata_AR_cash
account_receivables6 = revenue6 * rata_AR_cash

account_receivables7 = revenue7 * rata_AR_cash

In [18]:
Inventories1_persen = bs1['years2ago']['Inventories(%)']
Inventories2_persen = bs2['yearago']['Inventories(%)']
Inventories3_persen = bs3['lastyear']['Inventories(%)']
rata_Inventories = (Inventories1_persen + Inventories2_persen + Inventories3_persen)/3

inventories4 = revenue4 * rata_Inventories
inventories5 = revenue5 * rata_Inventories
inventories6 = revenue6 * rata_Inventories

In [19]:
account_Payable_1_persen = bs1['years2ago']['accounts Payable(%)']
account_Payable_2_persen = bs2['yearago']['accounts Payable(%)']
account_Payable_3_persen = bs3['lastyear']['accounts Payable(%)']
rata_Payable_cash = (account_Payable_1_persen + account_Payable_2_persen + account_Payable_3_persen)/3

account_Payable_4 = revenue4 * rata_Payable_cash
account_Payable_5 = revenue5 * rata_Payable_cash
account_Payable_6 = revenue6 * rata_Payable_cash

In [20]:
capital_Expenditure_1_persen = bs1['years2ago']['capital Expenditure(%)']
capital_Expenditure_2_persen = bs2['yearago']['capital Expenditure(%)']
capital_Expenditure_3_persen = bs3['lastyear']['capital Expenditure(%)']
rata_CE_cash = (capital_Expenditure_1_persen + capital_Expenditure_2_persen + capital_Expenditure_3_persen)/3

capital_Expenditure_4 = revenue4 * rata_CE_cash
capital_Expenditure_5 = revenue5 * rata_CE_cash
capital_Expenditure_6 = revenue6 * rata_CE_cash

In [21]:
bs4 = {
    'future1' : {
        'Total Cash': cash4, 
        'Total Cash(%)': rata2cash, 
        'account receivables' : account_receivables4, 
        'account receivables(%)' : rata_AR_cash, 
        'Inventories' : inventories4, 
        'Inventories(%)' : rata_Inventories, 
        'accounts Payable' : account_Payable_4, 
        'accounts Payable(%)' : rata_Payable_cash, 
        'capital Expenditure' : capital_Expenditure_4, 
        'capital Expenditure(%)' : rata_CE_cash 
    }
}

bs5 = {
    'future2' : {
        'Total Cash': cash5, 
        'Total Cash(%)': rata2cash, 
        'account receivables' : account_receivables5, 
        'account receivables(%)' : rata_AR_cash, 
        'Inventories' : inventories5, 
        'Inventories(%)' : rata_Inventories, 
        'accounts Payable' : account_Payable_5, 
        'accounts Payable(%)' : rata_Payable_cash, 
        'capital Expenditure' : capital_Expenditure_5, 
        'capital Expenditure(%)' : rata_CE_cash 
    }
}

bs6 = {
    'future3' : {
        'Total Cash': cash6, 
        'Total Cash(%)': rata2cash, 
        'account receivables' : account_receivables6, 
        'account receivables(%)' : rata_AR_cash, 
        'Inventories' : inventories6, 
        'Inventories(%)' : rata_Inventories, 
        'accounts Payable' : account_Payable_6, 
        'accounts Payable(%)' : rata_Payable_cash, 
        'capital Expenditure' : capital_Expenditure_6, 
        'capital Expenditure(%)' : rata_CE_cash 
    }
}

In [22]:
balance_sheet = bs1 | bs2 | bs3 | bs4 | bs5 | bs6
balance_sheet = pd.DataFrame.from_dict(balance_sheet,orient='columns')
balance_sheet

,years2ago,yearago,lastyear,future1,future2,future3
Total Cash,1.147960e+14,1.112278e+14,1.881546e+14,1.480345e+14,1.527335e+14,1.575818e+14
Total Cash(%),1.573231e+00,1.450045e+00,2.347825e+00,1.790367e+00,1.790367e+00,1.790367e+00
account receivables,2.967090e+11,4.071750e+11,6.067600e+11,4.670467e+11,4.818722e+11,4.971683e+11
account receivables(%),4.066270e-03,5.308224e-03,7.571253e-03,5.648582e-03,5.648582e-03,5.648582e-03
Inventories,1.036258e+12,1.528414e+12,1.401658e+12,1.422634e+12,1.467793e+12,1.514385e+12
Inventories(%),1.420147e-02,1.992550e-02,1.749012e-02,1.720570e-02,1.720570e-02,1.720570e-02
accounts Payable,7.067331e+14,8.455987e+14,9.802440e+14,9.078951e+14,9.367146e+14,9.664488e+14
accounts Payable(%),9.685476e+00,1.102383e+01,1.223165e+01,1.098032e+01,1.098032e+01,1.098032e+01
capital Expenditure,-2.675281e+12,-2.282555e+12,-3.061820e+12,-2.883641e+12,-2.975177e+12,-3.069619e+12
capital Expenditure(%),-3.666359e-02,-2.975702e-02,-3.820590e-02,-3.487550e-02,-3.487550e-02,-3.487550e-02


In [23]:
input_ticker = yf.Ticker(ticker_code)
info_ticker = input_ticker.info

In [24]:
#Weighted Average Cost Of Capital
share_price = info_ticker['regularMarketPrice']
beta = info_ticker['beta']
diluted_Shares_Outstanding = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualDilutedAverageShares'][-1]['reportedValue']['raw']
#tax_Rate = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualTaxRateForCalcs'][-1]['reportedValue']['raw']#*100
interestExpense = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualInterestExpense'][-1]['reportedValue']['raw']
totalDebt = B_S_start['QuoteTimeSeriesStore']['timeSeries']['annualTotalDebt'][-1]['reportedValue']['raw']
incomeTaxExp = I_S_start['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory'][0]['incomeTaxExpense']['raw']
incomeBeforeTax = I_S_start['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory'][0]['incomeBeforeTax']['raw']
riskFreeRate = I_S_start['StreamDataStore']['quoteData']['^TNX']['regularMarketPrice']['raw']


wacc = {}
wacc['Share Price'] = share_price
wacc['beta'] = beta
wacc['Diluted Shares Outstanding'] = diluted_Shares_Outstanding
#wacc['Tax Rate'] = tax_Rate
wacc['costOfDebt'] =  interestExpense/totalDebt
wacc['TaxRate'] = incomeTaxExp/incomeBeforeTax
wacc['After_tax_CostOfDebt'] = wacc['costOfDebt'] * (1 - wacc['TaxRate'])
if negara == 'indonesia':
    wacc['riskFreeRate'] = 5.5
else:
    wacc['riskFreeRate'] = riskFreeRate
wacc['marketriskpremium'] = 4.72
wacc['totaldebt'] = totalDebt
wacc['costofequity'] = wacc['riskFreeRate'] + (wacc['beta'] * wacc['marketriskpremium'])
wacc['totalequity'] = wacc['Share Price'] * wacc['Diluted Shares Outstanding']
wacc['totalcapital'] = wacc['totalequity'] + wacc['totaldebt']
wacc['DebtWeighting'] = (wacc['totaldebt'] / wacc['totalcapital'])*100
wacc['EquityWeighting'] = (wacc['totalequity'] / wacc['totalcapital'])*100
wacc['WACC'] = (wacc['EquityWeighting'] * wacc['costofequity'] + wacc['DebtWeighting'] * wacc['After_tax_CostOfDebt'])/100



In [25]:
table_wacc = pd.DataFrame.from_dict(wacc,orient='index')
table_wacc

,0
Share Price,8.375000e+03
beta,5.752130e-01
Diluted Shares Outstanding,1.232750e+11
costOfDebt,3.293362e+00
TaxRate,1.905456e-01
After_tax_CostOfDebt,2.665826e+00
riskFreeRate,5.500000e+00
marketriskpremium,4.720000e+00
totaldebt,2.289222e+12
costofequity,8.215005e+00


In [26]:
pretaxIncome = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualPretaxIncome']
TaxProvision = I_S_start['QuoteTimeSeriesStore']['timeSeries']['annualTaxProvision']

pretaxIncome_freecash_last = pretaxIncome[-1]['reportedValue']['raw']
TaxProvision_freecash_last = TaxProvision[-1]['reportedValue']['raw']
TaxRate_freecash_last = TaxProvision_freecash_last/pretaxIncome_freecash_last

pretaxIncome_freecash_1ago = pretaxIncome[-2]['reportedValue']['raw']
TaxProvision_freecash_1ago = TaxProvision[-2]['reportedValue']['raw']
TaxRate_freecash_1ago = TaxProvision_freecash_1ago/pretaxIncome_freecash_1ago

pretaxIncome_freecash_2ago = pretaxIncome[-3]['reportedValue']['raw']
TaxProvision_freecash_2ago = TaxProvision[-3]['reportedValue']['raw']
TaxRate_freecash_2ago = TaxProvision_freecash_2ago/pretaxIncome_freecash_2ago

taxRate_future = (TaxRate_freecash_last + TaxRate_freecash_1ago + TaxRate_freecash_2ago)/3

#Receivables = B_S_start['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
Receivables_freecash_1ago = balance_sheet['years2ago']['account receivables'] - balance_sheet['yearago']['account receivables'] #2019-2020
Receivables_freecash_last = balance_sheet['yearago']['account receivables'] - balance_sheet['lastyear']['account receivables'] #2020-2021

Receivables_future1 = balance_sheet['lastyear']['account receivables'] - balance_sheet['future1']['account receivables'] #2021-2022
Receivables_future2 = balance_sheet['future1']['account receivables'] - balance_sheet['future2']['account receivables'] #2022-2023
Receivables_future3 = balance_sheet['future2']['account receivables'] - balance_sheet['future3']['account receivables']

#====
ebiat_freecash_2ago = income_statement['years2ago']['Ebit']*(1-TaxRate_freecash_2ago)
ebiat_freecash_1ago = income_statement['yearago']['Ebit']*(1-TaxRate_freecash_1ago)
ebiat_freecash_last = income_statement['lastyear']['Ebit']*(1-TaxRate_freecash_last)
ebiat_freecash_future1 = ebit4*(1-(taxRate_future))
ebiat_freecash_future2 = ebit5*(1-(taxRate_future))
ebiat_freecash_future3 = ebit6*(1-(taxRate_future))

#====
depreciation_freecash_2ago = income_statement['years2ago']['Depreciation']
depreciation_year1ago = income_statement['yearago']['Depreciation']
depreciation_last = income_statement['lastyear']['Depreciation']
depreciation_future1 = income_statement['future1']['Depreciation']
depreciation_future2 = income_statement['future2']['Depreciation']
depreciation_future3 = income_statement['future3']['Depreciation']


#====
inventories_freecash_last = balance_sheet['yearago']['Inventories'] - balance_sheet['lastyear']['Inventories']
inventories_freecash_1ago = balance_sheet['years2ago']['Inventories'] - balance_sheet['yearago']['Inventories']
inventories_future1 = balance_sheet['lastyear']['Inventories'] - balance_sheet['future1']['Inventories']
inventories_future2 = balance_sheet['future1']['Inventories'] - balance_sheet['future2']['Inventories']
inventories_future3 = balance_sheet['future2']['Inventories'] - balance_sheet['future3']['Inventories']

#=====
accountsPayable_freecash_1ago = balance_sheet['yearago']['accounts Payable'] - balance_sheet['years2ago']['accounts Payable']
accountsPayable_freecash_last = balance_sheet['lastyear']['accounts Payable'] - balance_sheet['yearago']['accounts Payable']
accountsPayable_future1 = balance_sheet['future1']['accounts Payable'] - balance_sheet['lastyear']['accounts Payable']
accountsPayable_future2 = balance_sheet['future2']['accounts Payable'] - balance_sheet['future1']['accounts Payable']
accountsPayable_future3 = balance_sheet['future3']['accounts Payable'] - balance_sheet['future2']['accounts Payable']

#====
capitalExpenditure_freecash_2ago = balance_sheet['years2ago']['capital Expenditure']
capitalExpenditure_freecash_1ago = balance_sheet['yearago']['capital Expenditure']
capitalExpenditure_freecash_last = balance_sheet['lastyear']['capital Expenditure']

#UFCF
ufcf_2ago = ebiat_freecash_2ago + depreciation_freecash_2ago + capitalExpenditure_freecash_2ago
ufcf_1ago = ebiat_freecash_1ago + depreciation_year1ago + Receivables_freecash_1ago + inventories_freecash_1ago + accountsPayable_freecash_1ago + capitalExpenditure_freecash_1ago
ufcf_last = ebiat_freecash_last + depreciation_last + Receivables_freecash_last + inventories_freecash_last + accountsPayable_freecash_last + capitalExpenditure_freecash_last
ufcf_future1 = ebiat_freecash_future1 + depreciation_future1 + Receivables_future1 + inventories_future1 + accountsPayable_future1 + capital_Expenditure_4
ufcf_future2 = ebiat_freecash_future2 + depreciation_future2 + Receivables_future2 + inventories_future2 + accountsPayable_future2 + capital_Expenditure_5
ufcf_future3 = ebiat_freecash_future3 + depreciation_future3 + Receivables_future3 + inventories_future3 + accountsPayable_future3 + capital_Expenditure_5

#==== PV UFCF ====#
wacc_pv = table_wacc[0]['WACC']
pV_ufcf_future1 = ufcf_future1 / (1+(wacc_pv/100))
pV_ufcf_future2 = ufcf_future2 / (1+(wacc_pv/100))**2
pV_ufcf_future3 = ufcf_future3 / (1+(wacc_pv/100))**3

In [27]:

freecash= {

    'years2ago': {
                    'revenue': income_statement['years2ago']['revenue'],
                    'EBITDA' : income_statement['years2ago']['EBITDA'],
                    'Ebit' : income_statement['years2ago']['Ebit'],
                    'TaxRate': TaxRate_freecash_2ago*100,
                    'EBIAT' : ebiat_freecash_2ago,
                    'Depreciation' : depreciation_freecash_2ago,
                    'AccountsReceivable' : 0,
                    'Inventories' : 0,
                    'accountsPayable' : 0,
                    'Capital Expenditure' : capitalExpenditure_freecash_2ago,
                    'UFCF' : ufcf_2ago,
                },
    


    'yearago': {
                'revenue': income_statement['yearago']['revenue'],
                'EBITDA' : income_statement['yearago']['EBITDA'],
                'Ebit' : income_statement['yearago']['Ebit'],
                'TaxRate': TaxRate_freecash_1ago*100,
                'EBIAT' : ebiat_freecash_1ago,
                'Depreciation' : depreciation_year1ago,
                'AccountsReceivable' : Receivables_freecash_1ago,
                'Inventories' : inventories_freecash_1ago,
                'accountsPayable' : accountsPayable_freecash_1ago,
                'Capital Expenditure' : capitalExpenditure_freecash_1ago,
                'UFCF' : ufcf_1ago
            },
    

    'lastyear': {
                'revenue': income_statement['lastyear']['revenue'], 
                'EBITDA' : income_statement['lastyear']['EBITDA'], 
                'Ebit' : income_statement['lastyear']['Ebit'],
                'TaxRate': TaxRate_freecash_last*100,
                'EBIAT' : ebiat_freecash_last,
                'Depreciation' : depreciation_last,
                'AccountsReceivable' : Receivables_freecash_last,
                'Inventories' : inventories_freecash_last,
                'accountsPayable' : accountsPayable_freecash_last,
                'Capital Expenditure' : capitalExpenditure_freecash_last,
                'UFCF' : ufcf_last
            },
    

    'future1': {
                    'revenue': revenue4, 
                    'EBITDA' : ebitda4, 
                    'Ebit' : ebit4,
                    'TaxRate' : taxRate_future*100,
                    'EBIAT' : ebiat_freecash_future1,
                    'Depreciation' : depreciation_future1,
                    'AccountsReceivable' : Receivables_future1,
                    'Inventories' : inventories_future1,
                    'accountsPayable' : accountsPayable_future1,
                    'Capital Expenditure' : capital_Expenditure_4,
                    'UFCF' : ufcf_future1,
                    'PV_UFCF' : pV_ufcf_future1,                        
                },
    

    'future2': {
                    'revenue': revenue5, 
                    'EBITDA' : ebitda5, 
                    'Ebit' : ebit5,
                    'TaxRate' : taxRate_future*100,
                    'EBIAT' : ebiat_freecash_future2,
                    'Depreciation' : depreciation_future2,
                    'AccountsReceivable' : Receivables_future2,
                    'Inventories' : inventories_future2,
                    'accountsPayable' : accountsPayable_future2,
                    'Capital Expenditure' : capital_Expenditure_5,
                    'UFCF' : ufcf_future2,
                    'PV_UFCF' : pV_ufcf_future2,
                    
                },
    

    'future3': {
                    'revenue': revenue6, 
                    'EBITDA' : ebitda6, 
                    'Ebit' : ebit6,
                    'TaxRate' : taxRate_future*100,
                    'EBIAT' : ebiat_freecash_future3,
                    'Depreciation' : depreciation_future3,
                    'AccountsReceivable' : Receivables_future3,
                    'Inventories' : inventories_future3,
                    'accountsPayable' : accountsPayable_future3,
                    'Capital Expenditure' : capital_Expenditure_5,
                    'UFCF' : ufcf_future3,
                    'PV_UFCF' : pV_ufcf_future3,
                        
                }


}



In [28]:
freecash = pd.DataFrame.from_dict(freecash,orient='columns')
freecash
#ada perbedaan di AccountsReceivable

,years2ago,yearago,lastyear,future1,future2,future3
revenue,7.296834e+13,7.670644e+13,8.013998e+13,8.268387e+13,8.530851e+13,8.801647e+13
EBITDA,5.029426e+13,4.551218e+13,4.881010e+13,5.213635e+13,5.379132e+13,5.549883e+13
Ebit,4.860038e+13,4.327170e+13,4.667199e+13,4.995620e+13,5.154196e+13,5.317807e+13
TaxRate,2.127098e+01,1.912923e+01,1.905456e+01,1.981826e+01,1.981826e+01,1.981826e+01
EBIAT,3.826261e+13,3.499416e+13,3.777885e+13,4.005575e+13,4.132724e+13,4.263910e+13
Depreciation,1.693877e+12,2.240482e+12,2.138107e+12,2.180154e+12,2.249359e+12,2.320761e+12
AccountsReceivable,0.000000e+00,-1.104660e+11,-1.995850e+11,1.397133e+11,-1.482552e+10,-1.529613e+10
Inventories,0.000000e+00,-4.921560e+11,1.267560e+11,-2.097568e+10,-4.515884e+10,-4.659232e+10
accountsPayable,0.000000e+00,1.388656e+14,1.346453e+14,-7.234885e+13,2.881943e+13,2.973425e+13
Capital Expenditure,-2.675281e+12,-2.282555e+12,-3.061820e+12,-2.883641e+12,-2.975177e+12,-2.975177e+12


In [29]:
sum_pv_ufcf = pV_ufcf_future1 + pV_ufcf_future2 + pV_ufcf_future3
sum_pv_ufcf

85422267645834.97

In [30]:
if negara == 'indonesia':
    lgt = 1.902 #id
else:
    lgt = 2.8 #us

wacc_persen = table_wacc[0]['WACC']
fcf_t1 = (freecash['future3']['UFCF'] * (lgt/100)) + freecash['future3']['UFCF']
terminal_values = fcf_t1 / ((wacc_persen/100) - (lgt/100))
jumlah_future = 3
pv_terminal_value = terminal_values / ((1+(wacc_persen/100))**jumlah_future)

In [31]:
terminal_Value = {
    'lGT' : lgt,
    'wacc%' : wacc_persen,
    'fcf(t+1)' : fcf_t1,
    'terminal_values' : terminal_values,
    'pv_terminal_value' : pv_terminal_value
    
}

In [32]:
terminal_Value = pd.DataFrame.from_dict(terminal_Value,orient='index')
terminal_Value

,0
lGT,1.902000e+00
wacc%,8.202728e+00
fcf(t+1),7.301996e+13
terminal_values,1.158913e+15
pv_terminal_value,9.148212e+14


In [33]:
enterprise_Value = sum_pv_ufcf + terminal_Value[0]['pv_terminal_value']
totalDebt_a = table_wacc[0]['totaldebt']
annualCashAndCashEquivalents = B_S_start['QuoteTimeSeriesStore']['timeSeries']['annualCashAndCashEquivalents'][-1]['reportedValue']['raw']
try:
    net_debt = B_S_start['QuoteTimeSeriesStore']['timeSeries']['annualNetDebt'][-1]['reportedValue']['raw']
except:
    net_debt = totalDebt_a - annualCashAndCashEquivalents
equity_value = enterprise_Value - net_debt
share_outstanding = table_wacc[0]['Diluted Shares Outstanding']
equity_Value_PerShare = equity_value / share_outstanding

In [34]:
intrinsic_Value = {
    'Enterprise_Value' : enterprise_Value,
    'net_debt' : net_debt,
    'equity_value' : equity_value,
    'share_outstanding' : share_outstanding,
    'equity_Value_PerShare' : equity_Value_PerShare
}

In [35]:
intrinsic_Value = pd.DataFrame.from_dict(intrinsic_Value, orient='index')
intrinsic_Value

,0
Enterprise_Value,1.000243e+15
net_debt,-1.858654e+14
equity_value,1.186109e+15
share_outstanding,1.232750e+11
equity_Value_PerShare,9.621646e+03


In [36]:
intrinsic_Value[0]['equity_Value_PerShare']

9621.64586503493

In [37]:
pricess = table_wacc[0]['Share Price']
selisih = (equity_Value_PerShare - pricess)/pricess
selisih

0.1488532376161111

In [38]:
if selisih >= 0.50:
    nilai_wajar = (pricess*0.22)+pricess
else:
    nilai_wajar = equity_Value_PerShare

In [39]:
nilai_wajar

9621.64586503493

In [40]:
def fcf_levered(input1, fcf_start, ticker_code):
    if input1 == 'lastyear':
        year = 0
        input2 = -1
    elif input1 == 'yearago':
        year = 1
        input2 = -2
    elif input1 ==  'years2ago':
        year = 2
        input2 = -3
    elif input1 ==  'years3ago':
        year = 3
        input2 = -4
    elif input1 ==  'years4ago':
        year = 4
        input2 = -5
    else:
        print('out of index')
    
    try:
        operating_CashFlow = fcf_start['QuoteTimeSeriesStore']['timeSeries']['annualOperatingCashFlow'][input2]['reportedValue']['raw']
    except:
        operating_CashFlow = 0
    operating_CashFlow_persen = operating_CashFlow / income_statement[input1]['revenue']
    Capital_Expenditure = freecash[input1]['Capital Expenditure']
    Capital_Expenditure_persen = Capital_Expenditure / income_statement[input1]['revenue']
    freecashflow = fcf_start['QuoteTimeSeriesStore']['timeSeries']['annualFreeCashFlow'][input2]['reportedValue']['raw']
    
    
    fcf_lereved = {}
    fcf_lereved[input1] = {}
    fcf_lereved[input1]['operating_CashFlow'] = operating_CashFlow
    fcf_lereved[input1]['operating_CashFlow(%)'] = operating_CashFlow_persen
    fcf_lereved[input1]['Capital_Expenditure'] = Capital_Expenditure
    fcf_lereved[input1]['Capital_Expenditure(%)'] = Capital_Expenditure_persen
    fcf_lereved[input1]['freecashflow'] = freecashflow
    
    
    return fcf_lereved

In [41]:
fcf_lev_lastyear = fcf_levered('lastyear', fcf_start, ticker_code)
fcf_lev_yearago = fcf_levered('yearago', fcf_start, ticker_code)
fcf_lev_years2ago = fcf_levered('years2ago', fcf_start, ticker_code)

fcf_levered = fcf_lev_years2ago | fcf_lev_yearago | fcf_lev_lastyear

In [42]:
avg_operating_CashFlow_persen = (fcf_levered['years2ago']['operating_CashFlow(%)'] + fcf_levered['yearago']['operating_CashFlow(%)'] + fcf_levered['lastyear']['operating_CashFlow(%)'])/3
operating_CashFlow_future1 = avg_operating_CashFlow_persen * freecash['future1']['revenue']
operating_CashFlow_future2 = avg_operating_CashFlow_persen * freecash['future2']['revenue']
operating_CashFlow_future3 = avg_operating_CashFlow_persen * freecash['future3']['revenue']

avg_Capital_Expenditure_persen = (fcf_levered['years2ago']['Capital_Expenditure(%)'] + fcf_levered['yearago']['Capital_Expenditure(%)'] + fcf_levered['lastyear']['Capital_Expenditure(%)'])/3
Capital_Expenditure_future1 = avg_Capital_Expenditure_persen * freecash['future1']['revenue']
Capital_Expenditure_future2 = avg_Capital_Expenditure_persen * freecash['future2']['revenue']
Capital_Expenditure_future3 = avg_Capital_Expenditure_persen * freecash['future3']['revenue']

freecashflow1 = operating_CashFlow_future1 + Capital_Expenditure_future1
freecashflow2 = operating_CashFlow_future2 + Capital_Expenditure_future2
freecashflow3 = operating_CashFlow_future3 + Capital_Expenditure_future3

wacc_persen_L = table_wacc[0]['WACC']
pv_lereved_fcf1 = freecashflow1 / (1+(wacc_persen_L/100))
pv_lereved_fcf2 = freecashflow2 / (1+(wacc_persen_L/100))**2
pv_lereved_fcf3 = freecashflow3 / (1+(wacc_persen_L/100))**3

In [43]:
fcf_levered_future = {
    'future1' : { 
        'operating_CashFlow' : operating_CashFlow_future1,
        'operating_CashFlow(%)' : avg_operating_CashFlow_persen,
        'Capital_Expenditure' : Capital_Expenditure_future1,
        'Capital_Expenditure(%)' : avg_Capital_Expenditure_persen,
        'freecashflow' : freecashflow1,
        'wacc%' : wacc_persen_L,
        'PV_LFCF' : pv_lereved_fcf1
                },
    
    'future2' : { 
        'operating_CashFlow' : operating_CashFlow_future2,
        'operating_CashFlow(%)' : avg_operating_CashFlow_persen,
        'Capital_Expenditure' : Capital_Expenditure_future2,
        'Capital_Expenditure(%)' : avg_Capital_Expenditure_persen,
        'freecashflow' : freecashflow2,
        'wacc%' : wacc_persen_L,
        'PV_LFCF' : pv_lereved_fcf2
                },
    
    'future3' : { 
        'operating_CashFlow' : operating_CashFlow_future3,
        'operating_CashFlow(%)' : avg_operating_CashFlow_persen,
        'Capital_Expenditure' : Capital_Expenditure_future3,
        'Capital_Expenditure(%)' : avg_Capital_Expenditure_persen,
        'freecashflow' : freecashflow3,
        'wacc%' : wacc_persen_L,
        'PV_LFCF' : pv_lereved_fcf3
                },
}

In [44]:
fcf_levered = fcf_lev_years2ago | fcf_lev_yearago | fcf_lev_lastyear | fcf_levered_future

In [45]:
fcf_levered = pd.DataFrame.from_dict(fcf_levered,orient='columns')
fcf_levered

,years2ago,yearago,lastyear,future1,future2,future3
operating_CashFlow,5.194204e+13,5.097888e+13,1.261863e+14,8.133377e+13,8.391555e+13,8.657930e+13
operating_CashFlow(%),7.118435e-01,6.645970e-01,1.574574e+00,9.836715e-01,9.836715e-01,9.836715e-01
Capital_Expenditure,-2.675281e+12,-2.282555e+12,-3.061820e+12,-2.883641e+12,-2.975177e+12,-3.069619e+12
Capital_Expenditure(%),-3.666359e-02,-2.975702e-02,-3.820590e-02,-3.487550e-02,-3.487550e-02,-3.487550e-02
freecashflow,4.926676e+13,4.830514e+13,1.228809e+14,7.845012e+13,8.094038e+13,8.350968e+13
wacc%,NaN,NaN,NaN,8.202728e+00,8.202728e+00,8.202728e+00
PV_LFCF,NaN,NaN,NaN,7.250291e+13,6.913354e+13,6.592076e+13


In [46]:
sum_pv_lereved = pv_lereved_fcf1 + pv_lereved_fcf2 + pv_lereved_fcf3

In [47]:
fcf_lereved_p1 = (fcf_levered['future3']['freecashflow'] * (lgt/100)) + fcf_levered['future3']['freecashflow']

In [48]:
terminal_values_lereved = fcf_lereved_p1 / ((wacc_persen/100) - (lgt/100))

In [49]:
pv_terminal_value = terminal_values_lereved / ((1+(wacc_persen/100))**jumlah_future)

In [50]:
enterprise_value_lereved = sum_pv_lereved + pv_terminal_value
equity_value_lereved = enterprise_value_lereved - net_debt
equity_Value_PerShare_lereved = equity_value_lereved / share_outstanding

In [51]:
nilai_wajar_lereved = equity_Value_PerShare_lereved

In [52]:
print('DCF = ',nilai_wajar)
print('DCF Lereved = ', nilai_wajar_lereved)

DCF =  9621.64586503493
DCF Lereved =  11839.884487887803
